# Presenting PISA results
## 0. Warming up 
### Reading the data 
- From Excel file riket2023_åk9_np.xlsx"
- for different sheets representing the sujects

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
def read_data(mysheetname):
    df_tmp=pd.read_excel("./Data/riket2023_åk9_np.xlsx", sheet_name=mysheetname, skiprows=8)
    df_tmp.columns=['Plats', 'Huvudman','Totalt (A-F)','Flickor (A-F)','Pojkar (A-F)','Totalt (A-E)','Flickor (A-E)','Pojkar (A-E)','Totalt (poäng)','Flickor (poäng)','Pojkar (poäng)']
    return df_tmp

df_engelska=read_data("Engelska")
df_matematik=read_data("Matematik")
df_matematik.strip('~')
df_svenska=read_data("Svenska")
df_sv_andra_sp=read_data("Svenska som andraspråk")

AttributeError: 'DataFrame' object has no attribute 'strip'

### How man has failed mathematics

In [27]:
def func_failed(row):
    z = row['Totalt (A-F)']*(1-row['Totalt (A-E)']/100)
    return z


df_matematik.apply(func_failed, axis=1)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [36]:
df_matematik['Totalt (A-E)']

KeyError: '~100'